In [1]:
from structure import World
import pandas as pd
import numpy as np
from streamz.dataframe import PeriodicDataFrame
from time import sleep
from IPython.display import display
import ipywidgets as widgets
from threading import Thread
from ipywidgets import interact

In [2]:
def get_global_report(results):
    report = pd.DataFrame({'n_colonies': [0 for _ in range(n_worlds+1)],
                           'n_ants': [0 for _ in range(n_worlds+1)], 
                           'n_ants_with_food': [0 for _ in range(n_worlds+1)], 
                           'n_food_in_colonies': [0 for _ in range(n_worlds+1)], 
                           'n_food_in_foods': [0 for _ in range(n_worlds+1)]},
                           index=pd.Index(data=[i for i in range(n_worlds)] + ['total'], name='world'))

    for world_num, result in enumerate(results):
        winner, x, y, food_results, colony_results = result

        for col_num, x, y, amount_of_food, ant_results, pheromone_results in colony_results:
            report['n_colonies'][world_num] += 1
            report['n_food_in_colonies'][world_num] += amount_of_food

            for ant_num, x, y, old_x, old_y, has_food, punctuation in ant_results:
                report['n_ants'][world_num] += 1

                if has_food:
                    report['n_ants_with_food'][world_num] += 1

        for num, x, y, amount, inside, limit in food_results:
            report['n_food_in_foods'][world_num] += amount
            
    report.loc['total'] = report.iloc[:n_worlds].sum(axis=0)
    
    return report

In [3]:
def get_world_report(result):
    winner, x, y, food_results, colony_results = result

    n_colonies = len(colony_results)
    report = pd.DataFrame({'n_ants': [0 for _ in range(n_colonies+1)],
                           'n_food': [0 for _ in range(n_colonies+1)]},
                           index=pd.Index(data=[i for i in range(n_colonies)] + ['total'], name='colony'))

    for col_num, x, y, amount_of_food, ant_results, pheromone_results in colony_results:
        for ant_num, x, y, old_x, old_y, has_food, punctuation in ant_results:
            report['n_ants'][col_num] += 1

        report['n_food'][col_num] = amount_of_food

    report.loc['total'] = report.iloc[:n_colonies].sum(axis=0)
    
    return report

In [4]:
sleeptime = 0.5
n_worlds = 5
report_type = 'Global'

In [5]:
report = pd.DataFrame()
view = PeriodicDataFrame(lambda **kwargs: report, interval=sleeptime)

In [6]:
widget = widgets.Dropdown(
    options=['Global'] + [f'World {i}' for i in range(n_worlds)],
    value='Global',
    description='View'
)

def on_change(change):
    global report_type
    
    if change['type'] == 'change' and change['name'] == 'value':
        report_type = change['new']

widget.observe(on_change)
    
def display_widget():
    display(widget)
    display(view)

In [7]:
def run_worlds(n=1000):
    global report, report_type
    
    worlds = [World(1000) for _ in range(n_worlds)]

    for _ in range(n):
        results = [world.update() for world in worlds]

        if report_type == 'Global':
            report = get_global_report(results)
        else:
            report = get_world_report(results[int(report_type.split()[1])])

        sleep(sleeptime)

In [8]:
Thread(target = run_worlds).start()
display_widget()

Dropdown(description='View', options=('Global', 'World 0', 'World 1', 'World 2', 'World 3', 'World 4'), value=…

Output()